In [1]:
TRAIN_ALL = False
TRAIN_BEST = True

In [2]:
from IPython.display import display, Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.utils import class_weight

from keras.callbacks import EarlyStopping
from keras.layers import InputLayer, BatchNormalization, Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential, load_model, model_from_json
from keras.utils import plot_model, to_categorical

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

import itertools
import keras
import time
import os

Using TensorFlow backend.


## Utilities

In [3]:
def plot_confusion_matrix(cm, class_labels, model_name, cmap=plt.cm.Blues):
    diagram_dir = os.path.join(os.getcwd(), "images")
    if not os.path.exists(diagram_dir):
        os.makedirs(diagram_dir)
    
    stats_dir = os.path.join(os.getcwd(), "stats")
    if not os.path.exists(stats_dir):
        os.makedirs(stats_dir)
    
    # create the figure
    fig = plt.figure()
    fig.set_size_inches(12,10)
    
    # labels
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    
    # tick marks
    class_count = len(class_labels)
    tick_marks = np.arange(class_count + 1)
    plt.xticks(tick_marks, class_labels, rotation=45)
    plt.yticks(tick_marks, class_labels)
    
    # normalization
    cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    cm = np.around(cm, 6)
    cm = np.nan_to_num(cm)

    # numbers
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:.4f}".format(cm[i, j]), horizontalalignment="center", color="white" if cm[i,j] > thresh else "black")
    
    # color bar
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.colorbar()
    
    # save and show
    np.set_printoptions(formatter={"float": lambda x: "{0:0.4f}".format(x)})
    plt.savefig(os.path.join(diagram_dir, model_name + "_confusion_matrix.png"), dpi=100)
    pd.DataFrame(cm).to_csv(os.path.join(stats_dir, model_name + "_confusion_matrix.txt"), header=None, index=None)
    plt.show(block=False)
    

def graph_history(history, model_name, validation_data=False):
    directory = os.path.join(os.getcwd(), "images")
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # summarize history for accuracy
    fig = plt.figure()
    plt.plot(history.history["acc"], linestyle="-", color="k")
    
    if validation_data:
        plt.plot(history.history["val_acc"], linestyle="--", color="grey")
        plt.legend(["Tr. Acc", "Val. Acc"])
    
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.savefig(os.path.join(directory, model_name + "_history_accuracy.png"), dpi=400)
    plt.show()
 
    # summarize history for loss
    fig = plt.figure()
    plt.plot(history.history["loss"], linestyle="-", color="k")
    
    if validation_data:
        plt.plot(history.history["val_loss"], linestyle="--", color="grey")
        plt.legend(["Tr. Loss", "Val. Loss"])
    
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.savefig(os.path.join(directory, model_name + "_history_loss.png"), dpi=400)
    plt.show()

    
def save_model(model, model_name):
    directory = os.path.join(os.getcwd(), "models")
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # save architecture
    with open(os.path.join(directory, model_name + ".json"), "w+") as f:
        f.write(model.to_json())
    
    # save weights
    model.save_weights(os.path.join(directory, model_name + "_weights.h5"))

def load_model(model_name):
    directory = os.path.join(os.getcwd(), "models")

    if os.path.exists(directory):
        # load the architecture
        json_file = open(os.path.join(directory, model_name + ".json"))
        model = model_from_json(json_file.read())
        model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
        json_file.close()

        # load the weights and return the model
        model.load_weights(os.path.join(directory, model_name + "_weights.h5"))
        return model

## Load data

In [4]:
# how long does this take?
start = time.time()

filename = "/Users/david/Documents/masters/thesis/data/snapshots/unfiltered_snapshots.csv"

# read the file with all dtypes
col_names = pd.read_csv(filename, nrows=0).columns
types_dict = {"Vehicle": int, "Capture": int}
types_dict.update({col: int for col in col_names if col not in types_dict})
df = pd.read_csv(filename, dtype=types_dict)

display(df.sample(frac=0.0001).head(5))
print("Elapsed time: {:.2f} seconds".format(time.time() - start))

,Vehicle,Capture,Data 1,Data 2,Data 3,Data 4,Data 5,Data 6,Data 7,Data 8,...,Data 1015,Data 1016,Data 1017,Data 1018,Data 1019,Data 1020,Data 1021,Data 1022,Data 1023,Data 1024
275761,102,102,22,246,51,26,65,8,0,7,...,127,15,239,0,12,0,9,0,0,64
152736,3,17,0,0,0,1,190,13,220,0,...,0,0,0,0,81,8,21,0,64,0
135420,3,16,255,0,255,255,127,232,2,24,...,218,0,112,14,0,0,0,80,101,126
249563,7,30,0,0,0,0,0,0,0,0,...,29,234,0,0,255,0,0,63,239,254
211763,5,21,160,2,1,10,0,0,0,0,...,0,0,0,0,0,0,0,27,44,3


Elapsed time: 38.79 seconds


In [5]:
# relabel the vehicles
new_labels = {v: i for i, v in enumerate(df["Vehicle"].unique())}
old_labels = {i: v for i, v in enumerate(df["Vehicle"].unique())}

df["Vehicle"] = df["Vehicle"].map(new_labels)
display(df.sample(frac=0.0001).head(5))

,Vehicle,Capture,Data 1,Data 2,Data 3,Data 4,Data 5,Data 6,Data 7,Data 8,...,Data 1015,Data 1016,Data 1017,Data 1018,Data 1019,Data 1020,Data 1021,Data 1022,Data 1023,Data 1024
243871,6,29,137,160,129,235,0,0,0,0,...,130,39,7,254,7,254,17,117,0,0
253478,6,31,32,0,0,0,0,0,0,0,...,0,0,255,254,126,242,235,164,48,0
165725,3,20,2,7,0,49,128,0,132,84,...,0,0,4,96,0,0,0,0,0,0
154125,3,18,3,127,1,34,0,0,0,0,...,57,113,82,19,26,204,33,251,65,242
164042,3,20,7,235,0,49,128,16,128,20,...,247,255,96,0,127,244,0,0,0,0


## Separate the data

In [6]:
"""
All vehicles known
"""

# split into (x, y) sets
all_df_x = df.drop(columns=["Vehicle", "Capture"])
all_df_y = df["Vehicle"]

# split known vehicles into train/validation/test (60/20/20)
all_nontest_x, all_test_x, all_nontest_y, all_test_y = train_test_split(all_df_x, all_df_y, test_size=0.2)
all_train_x, all_val_x, all_train_y, all_val_y = train_test_split(all_nontest_x, all_nontest_y, test_size=0.25)

# scale the inputs
all_train_x /= 255.
all_val_x /= 255.
all_test_x /= 255.

# add the extra dimensions
all_train_x = np.expand_dims(all_train_x, axis=2)
all_val_x = np.expand_dims(all_val_x, axis=2)
all_test_x = np.expand_dims(all_test_x, axis=2)

# encode the labels as one-hot variables
all_train_y = keras.utils.to_categorical(all_train_y, 20)
all_val_y = keras.utils.to_categorical(all_val_y, 20)
all_test_y = keras.utils.to_categorical(all_test_y, 20)

# sanity check
print("Train      : {} {}".format(all_train_x.shape, all_train_y.shape))
print("Validation : {} {}".format(all_val_x.shape, all_val_y.shape))
print("Test       : {} {}".format(all_test_x.shape, all_test_y.shape))

Train      : (179229, 1024, 1) (179229, 20)
Validation : (59743, 1024, 1) (59743, 20)
Test       : (59743, 1024, 1) (59743, 20)


In [7]:
"""
All vehicles balanced
"""

# balance the dataset
min_size = np.min([len(df[df["Vehicle"] == i]) for i in df["Vehicle"].unique()])
balanced_df = df.groupby("Vehicle", group_keys=False).apply(pd.DataFrame.sample, n=min_size)

# split into (x, y) sets
balanced_df_x = balanced_df.drop(columns=["Vehicle", "Capture"])
balanced_df_y = balanced_df["Vehicle"]

# split known vehicles into train/validation/test (60/20/20)
balanced_nontest_x, balanced_test_x, balanced_nontest_y, balanced_test_y = train_test_split(balanced_df_x, balanced_df_y, test_size=0.2)
balanced_train_x, balanced_val_x, balanced_train_y, balanced_val_y = train_test_split(balanced_nontest_x, balanced_nontest_y, test_size=0.25)

# scale the inputs
balanced_train_x /= 255.
balanced_val_x /= 255.
balanced_test_x /= 255.

# add the extra dimensions
balanced_train_x = np.expand_dims(balanced_train_x, axis=2)
balanced_val_x = np.expand_dims(balanced_val_x, axis=2)
balanced_test_x = np.expand_dims(balanced_test_x, axis=2)

# encode the labels as one-hot variables
balanced_train_y = keras.utils.to_categorical(balanced_train_y, 20)
balanced_val_y = keras.utils.to_categorical(balanced_val_y, 20)
balanced_test_y = keras.utils.to_categorical(balanced_test_y, 20)

# sanity check
print("Train      : {} {}".format(balanced_train_x.shape, balanced_train_y.shape))
print("Validation : {} {}".format(balanced_val_x.shape, balanced_val_y.shape))
print("Test       : {} {}".format(balanced_test_x.shape, balanced_test_y.shape))

Train      : (20508, 1024, 1) (20508, 20)
Validation : (6836, 1024, 1) (6836, 20)
Test       : (6836, 1024, 1) (6836, 20)


In [8]:
"""
Only Stone's vehicles
"""

# split into (x, y) sets
stone = df[df["Vehicle"] >= 9]
stone_x = stone.drop(columns=["Vehicle", "Capture"])
stone_y = stone["Vehicle"] - 9

# split known vehicles into train/validation/test (60/20/20)
stone_nontest_x, stone_test_x, stone_nontest_y, stone_test_y = train_test_split(stone_x, stone_y, test_size=0.2)
stone_train_x, stone_val_x, stone_train_y, stone_val_y = train_test_split(stone_nontest_x, stone_nontest_y, test_size=0.25)

# scale the inputs
stone_train_x /= 255.
stone_val_x /= 255.
stone_test_x /= 255.

# add the extra dimensions
stone_train_x = np.expand_dims(stone_train_x, axis=2)
stone_val_x = np.expand_dims(stone_val_x, axis=2)
stone_test_x = np.expand_dims(stone_test_x, axis=2)

# encode the labels as one-hot variables
stone_train_y = keras.utils.to_categorical(stone_train_y, 11)
stone_val_y = keras.utils.to_categorical(stone_val_y, 11)
stone_test_y = keras.utils.to_categorical(stone_test_y, 11)

# sanity check
print("Train      : {} {}".format(stone_train_x.shape, stone_train_y.shape))
print("Validation : {} {}".format(stone_val_x.shape, stone_val_y.shape))
print("Test       : {} {}".format(stone_test_x.shape, stone_test_y.shape))

Train      : (17055, 1024, 1) (17055, 11)
Validation : (5685, 1024, 1) (5685, 11)
Test       : (5685, 1024, 1) (5685, 11)


In [9]:
"""
Stone's vehicles balanced
"""

# balance the dataset
stone_min_size = np.min([len(stone[stone["Vehicle"] == i]) for i in stone["Vehicle"].unique()])
stone_balanced_df = stone.groupby("Vehicle", group_keys=False).apply(pd.DataFrame.sample, n=stone_min_size)

# split into (x, y) sets
stone_balanced_df_x = balanced_df.drop(columns=["Vehicle", "Capture"])
stone_balanced_df_y = balanced_df["Vehicle"] - 9

# split vehicles into train/validation/test (60/20/20)
stone_balanced_nontest_x, stone_balanced_test_x, stone_balanced_nontest_y, stone_balanced_test_y \
    = train_test_split(stone_balanced_df_x, stone_balanced_df_y, test_size=0.2)
stone_balanced_train_x, stone_balanced_val_x, stone_balanced_train_y, stone_balanced_val_y \
    = train_test_split(stone_balanced_nontest_x, stone_balanced_nontest_y, test_size=0.25)

# scale the inputs
stone_balanced_train_x /= 255.
stone_balanced_val_x /= 255.
stone_balanced_test_x /= 255.

# add the extra dimensions
stone_balanced_train_x = np.expand_dims(stone_balanced_train_x, axis=2)
stone_balanced_val_x = np.expand_dims(stone_balanced_val_x, axis=2)
stone_balanced_test_x = np.expand_dims(stone_balanced_test_x, axis=2)

# encode the labels as one-hot variables
stone_balanced_train_y = keras.utils.to_categorical(stone_balanced_train_y, 11)
stone_balanced_val_y = keras.utils.to_categorical(stone_balanced_val_y, 11)
stone_balanced_test_y = keras.utils.to_categorical(stone_balanced_test_y, 11)

# sanity check
print("Train      : {} {}".format(stone_balanced_train_x.shape, stone_balanced_train_y.shape))
print("Validation : {} {}".format(stone_balanced_val_x.shape, stone_balanced_val_y.shape))
print("Test       : {} {}".format(stone_balanced_test_x.shape, stone_balanced_test_y.shape))

Train      : (20508, 1024, 1) (20508, 11)
Validation : (6836, 1024, 1) (6836, 11)
Test       : (6836, 1024, 1) (6836, 11)


## Build the models

In [19]:
def build_model(num_filters, kernel_size, pool_size, filename):
    directory = os.path.join(os.getcwd(), "images")
    if not os.path.exists(directory):
        os.makedirs(directory)

    model = Sequential()
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation="relu", input_shape=(1024,1)))
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(100, activation="relu"))
    model.add(Dense(11, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

    print(model.summary())
    filename = os.path.join(directory, filename + "_diagram.pdf")
    plot_model(model, show_shapes=True, show_layer_names=False, to_file=filename)
    # display(Image(filename=filename, width=400))
    
    return model

In [20]:
# model configurations
num_filters = [32, 64]
kernel_size = [2, 3, 4]
pool_size = [2, 3, 4]
datasets = ["all"]

## Train and test all models

In [21]:
if TRAIN_ALL:
    # how long does this take?
    start_time = time.time()
    
    # we'll need these
    train_x = stone_train_x
    train_y = stone_train_y
    val_x = stone_val_x
    val_y = stone_val_y
    test_x = stone_test_x
    test_y = stone_test_y

    # gather training stats
    stats_dir = os.path.join(os.getcwd(), "stats")
    if not os.path.exists(stats_dir):
        os.makedirs(stats_dir)

    stats_path = os.path.join(stats_dir, "training_stats.csv")
    stats_file = open(stats_path, "w+")
    stats_file.write(
        "Model,Parameters,Training Epoch,Training Loss,Training Accuracy,Test Loss,Test Accuracy,Balanced Test Accuracy\n")

    # gather model stats
    models_dir = os.path.join(os.getcwd(), "models")
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    models_path = os.path.join(models_dir, "models.csv")
    models_file = open(models_path, "w+")
    models_file.write("Model,Filters,Kernel Size,Pooling Size\n")

    # stop early if possible
    callbacks = [EarlyStopping(
        monitor = "val_loss",
        patience = 10,
        restore_best_weights = True
    )]

    # this'll take a while
    count = 0
    for n in num_filters:
        for k in kernel_size:
            for p in pool_size:
                print("{}\n{} Model {}: filters = {}, kernel size = {}, pooling size = {}\n{}".format(
                    "=" * 100, "=" * 5, count, n, k, p, "=" * 100))

                # filepaths
                model_name = "model_{}".format(count)

                # build and show model
                model = build_model(n, k, p, model_name)

                # we'll need these these
                y_ints = [y.argmax() for y in train_y]
                class_weights = class_weight.compute_class_weight("balanced", np.unique(y_ints), y_ints)

                # train model
                history = model.fit(
                    train_x, train_y,
                    validation_data = (val_x, val_y),
                    class_weight = class_weights,
                    batch_size = 512,
                    epochs = 100,
                    shuffle = True,
                    verbose = 1,
                    callbacks = callbacks)

                # save trained model
                save_model(model, model_name)

                # show the graph history
                graph_history(history, model_name, validation_data=True)

                # evaluate the model
                evaluation = model.evaluate(test_x, test_y, verbose=0)

                print("--- Test set evaluation ---")
                print("Loss              : {:.4f}".format(evaluation[0]))
                print("Accuracy          : {:.4f}".format(evaluation[1]))

                # predict the test set and save the labels
                actual_labels = np.argmax(test_y, axis=1)
                predicted_labels = np.argmax(model.predict(test_x), axis=1)
                balanced_acc = balanced_accuracy_score(actual_labels, predicted_labels)

                print("Balanced accuracy : {:.4f}".format(balanced_acc))

                # revert to the original vehicle IDs
                actual_labels = [old_labels[i] for i in actual_labels]
                predicted_labels = [old_labels[i] for i in predicted_labels]

                # generate, save, and display confusion matrix
                cm = confusion_matrix(predicted_labels, actual_labels)
                plot_confusion_matrix(cm, class_labels=old_labels.values(), model_name=model_name)

                # get stats
                params = model.count_params()
                best_epoch = np.argmin(history.history["val_loss"])
                best_loss = history.history["val_loss"][best_epoch]
                best_acc = history.history["val_acc"][best_epoch]

                # save stats
                stats = [count, params, best_epoch + 1, best_loss, best_acc, evaluation[0], evaluation[1], balanced_acc]
                stats_file.write(",".join([str(s) for s in stats]) + "\n")

                # save model metadata
                model_metadata = [count, n, k, p]
                models_file.write(",".join([str(d) for d in model_metadata]) + "\n")

                # next model
                count += 1

    stats_file.close()
    models_file.close()

    elapsed_time = time.time() - start_time
    print("Elapsed time: {:.2f} seconds ({:.2f} hours)".format(elapsed_time, elapsed_time / 3600.))

## Train and test the best model

In [22]:
if TRAIN_BEST:
#     for dataset in ["all", "balanced"]:
    for dataset in ["all"]:
#     for dataset in ["balanced"]:
        # best model
        training_stats = pd.read_csv(os.path.join(os.getcwd(), "stats", "training_stats.csv"))
        best_model = int(training_stats.sort_values(by=["Training Loss"]).iloc[0]["Model"])
        best_epoch = int(training_stats.sort_values(by=["Training Loss"]).iloc[0]["Training Epoch"])
        
        # model parameters
        model_params = pd.read_csv(os.path.join(os.getcwd(), "models", "models.csv"))
        best_model = model_params.iloc[best_model]
        print(best_model)

        # filepaths
        model_name = "final_model_all" if dataset == "all" else "final_model_balanced"

        # build and show model
        model = build_model(int(best_model["Filters"]), int(best_model["Kernel Size"]), int(best_model["Pooling Size"]), model_name)

#         # we'll need these
#         train_x = stone_train_x if dataset == "all" else stone_balanced_train_x
#         train_y = stone_train_y if dataset == "all" else stone_balanced_train_y
#         val_x = stone_val_x if dataset == "all" else stone_balanced_val_x
#         val_y = stone_val_y if dataset == "all" else stone_balanced_val_y
#         test_x = stone_test_x if dataset == "all" else stone_balanced_test_x
#         test_y = stone_test_y if dataset == "all" else stone_balanced_test_y

#         # and these
#         y_ints = [y.argmax() for y in train_y]
#         class_weights = class_weight.compute_class_weight("balanced", np.unique(y_ints), y_ints)
#         class_weights = class_weights if dataset == "all" else None
        
#         history = model.fit(
#             np.concatenate((train_x, val_x)), np.concatenate((train_y, val_y)),
#             class_weight = class_weights,
#             batch_size = 512,
#             epochs = best_epoch,
#             shuffle = True,
#             verbose = 1)

#         # save trained model
#         save_model(model, model_name)

#         # show the graph history
#         graph_history(history, model_name)

#         # evaluate the model
#         evaluation = model.evaluate(test_x, test_y, verbose=0)

#         print("--- Test set evaluation ---")
#         print("Loss              : {:.4f}".format(evaluation[0]))
#         print("Accuracy          : {:.4f}".format(evaluation[1]))

#         # predict the test set and save the labels
#         actual_labels = np.argmax(test_y, axis=1)
#         predicted_labels = np.argmax(model.predict(test_x), axis=1)

#         print("Balanced accuracy : {:.4f}".format(balanced_accuracy_score(actual_labels, predicted_labels)))

#         # revert to the original vehicle IDs
#         actual_labels = [old_labels[i] for i in actual_labels]
#         predicted_labels = [old_labels[i] for i in predicted_labels]

#         # generate, save, and display confusion matrix
#         cm = confusion_matrix(predicted_labels, actual_labels)
#         plot_confusion_matrix(cm, class_labels=old_labels.values(), model_name=model_name)

Model            8
Filters         32
Kernel Size      4
Pooling Size     4
Name: 8, dtype: int64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5_input (InputLayer)  (None, 1024, 1)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1021, 32)          160       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1018, 32)          4128      
_________________________________________________________________
batch_normalization_3 (Batch (None, 1018, 32)          128       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 254, 32)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 254, 32)           0         
____________________________________________

## Load and test the best model

In [18]:
# if TRAIN_BEST:
#     for dataset in ["all", "balanced"]:
#         # we'll need these
#         test_x = stone_test_x if dataset == "all" else stone_balanced_test_x
#         test_y = stone_test_y if dataset == "all" else stone_balanced_test_y

#         # name the model
#         model_name = "final_model_all" if dataset == "all" else "final_model_balanced"
#         model = load_model(model_name)

#         # evaluate the model
#         evaluation = model.evaluate(test_x, test_y, verbose=0)

#         print("--- Test set evaluation ---")
#         print("Loss              : {:.4f}".format(evaluation[0]))
#         print("Accuracy          : {:.4f}".format(evaluation[1]))

#         # predict the test set and save the labels
#         actual_labels = np.argmax(test_y, axis=1)
#         predicted_labels = np.argmax(model.predict(test_x), axis=1)

#         print("Balanced accuracy : {:.4f}".format(balanced_accuracy_score(actual_labels, predicted_labels)))

#         # revert to the original vehicle IDs
#         actual_labels = [old_labels[i] for i in actual_labels]
#         predicted_labels = [old_labels[i] for i in predicted_labels]

#         # generate, save, and display confusion matrix
#         cm = confusion_matrix(predicted_labels, actual_labels)
#         plot_confusion_matrix(cm, class_labels=old_labels.values(), model_name=model_name)